## Notebook to process data from woobesluiten.csv.gz 

#### What does it do?
- Picks out decisions made only by ministeries in 2023
- Extracts decision dates with regex where possible
- Further tries to extract decision dates by looking at the sentence where 'Datum' is first mentioned
- Saves the DataFrame to "data/ministeries2023extracted.pkl.gz" in a pickle file to preserve datatypes and indexes

In [11]:
# Imports
import re
import spacy
import string
import numpy as np
import pandas as pd
import dateparser
from datetime import datetime, timezone
import pickle

from config import start_date, end_date, save_path

In [17]:
woobesluiten_path = 'data/woobesluiten.csv.gz'
current_save_path = 'data/minextracted.pkl.gz'
print(f"Extracting data from woo-dossiers from {start_date} to {end_date}")
print(f"When done it will be saved to: {save_path}")

Extracting data from woo-dossiers from 2019-04-01 to 2024-01-10
When done it will be saved to: data/minfinal.pkl.gz


In [18]:
woobesluiten = pd.read_csv(woobesluiten_path)

nlp = spacy.load("nl_core_news_lg")
pd.options.mode.chained_assignment = None

C:\Users\Joran\AppData\Local\Temp\ipykernel_26320\3620951926.py:1: DtypeWarning: Columns (7,15,16,17,18,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  woobesluiten = pd.read_csv(woobesluiten_path)


In [19]:
woobesluiten

,foi_dossierId,foi_pageNumber,dc_source,foi_decisionDate,dc_title,dc_description,foi_fileName,dc_format,dc_type,foi_nrPages,...,foi_nrRedactedRegions,foi_contourArea,foi_textArea,foi_charArea,foi_percentageTextAreaRedacted,foi_percentageCharAreaRedacted,foi_imageArea,foi_imageCoversFullPage,foi_bodyTextJaccard,foi_documentId
0,https://doi.wooverheid.nl/?doi=nl.mnre1010.2i....,1.0,https://open.overheid.nl/Details/ronl-decc6e99...,2023-03-20,Besluit op Woo-verzoek over verschillende stra...,NaN,Besluit op Woo-verzoek over verschillende stra...,NaN,besluit,2.0,...,6.0,190660.0,5766300.0,1204577.0,3.31,15.83,500990.0,1.0,0.918239,nl.mnre1010.2i.2023.1.doc.1
1,https://doi.wooverheid.nl/?doi=nl.mnre1010.2i....,2.0,https://open.overheid.nl/Details/ronl-decc6e99...,2023-03-20,Besluit op Woo-verzoek over verschillende stra...,NaN,Besluit op Woo-verzoek over verschillende stra...,NaN,besluit,2.0,...,1.0,109093.0,1325436.0,462155.0,8.23,23.61,500990.0,1.0,0.939759,nl.mnre1010.2i.2023.1.doc.1
2,https://doi.wooverheid.nl/?doi=nl.mnre1010.2i....,1.0,https://open.overheid.nl/Details/ronl-79e28a77...,2023-03-21,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,besluit,128.0,...,3.0,374734.5,5222535.0,1566908.5,7.18,23.92,494543.0,1.0,0.974619,nl.mnre1010.2i.2023.10.doc.1
3,https://doi.wooverheid.nl/?doi=nl.mnre1010.2i....,2.0,https://open.overheid.nl/Details/ronl-79e28a77...,2023-03-21,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,besluit,128.0,...,0.0,0.0,3668285.0,1568445.0,0.00,0.00,493896.0,1.0,1.000000,nl.mnre1010.2i.2023.10.doc.1
4,https://doi.wooverheid.nl/?doi=nl.mnre1010.2i....,3.0,https://open.overheid.nl/Details/ronl-79e28a77...,2023-03-21,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,Tweede deelbesluit op Woo-verzoek over tijdeli...,NaN,besluit,128.0,...,0.0,0.0,3733344.0,1723576.0,0.00,0.00,493749.0,1.0,0.978723,nl.mnre1010.2i.2023.10.doc.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37839,https://doi.wooverheid.nl/?doi=nl.mnre1153.2i....,3.0,https://open.overheid.nl/Details/ronl-7348da1c...,2023-03-08,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,besluit,7.0,...,2.0,97308.5,5496538.0,707907.5,1.77,13.75,500990.0,1.0,0.000000,nl.mnre1153.2i.2023.98.doc.1
37840,https://doi.wooverheid.nl/?doi=nl.mnre1153.2i....,4.0,https://open.overheid.nl/Details/ronl-7348da1c...,2023-03-08,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,besluit,7.0,...,1.0,8849.5,4839992.0,1368589.5,0.18,0.65,500990.0,1.0,0.000000,nl.mnre1153.2i.2023.98.doc.1
37841,https://doi.wooverheid.nl/?doi=nl.mnre1153.2i....,5.0,https://open.overheid.nl/Details/ronl-7348da1c...,2023-03-08,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,besluit,7.0,...,0.0,0.0,2942485.0,1363624.0,0.00,0.00,0.0,0.0,0.987013,nl.mnre1153.2i.2023.98.doc.1
37842,https://doi.wooverheid.nl/?doi=nl.mnre1153.2i....,6.0,https://open.overheid.nl/Details/ronl-7348da1c...,2023-03-08,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,Besluit op Woo-verzoek over meitellingen en Ie...,NaN,besluit,7.0,...,2.0,221473.0,5548085.0,518613.0,3.99,42.70,500990.0,1.0,0.000000,nl.mnre1153.2i.2023.98.doc.1


In [20]:
# multi index
woobesluiten.set_index(['foi_documentId', 'foi_pageNumber'], inplace=True)

In [21]:
# alleen ministeries overhouden
wooMinisterie = woobesluiten[woobesluiten['dc_publisher_name'].str.contains('Ministerie')]
wooMinisterie.shape

(37844, 50)

In [25]:
# ministeries van 2023
wooMinisterie['foi_decisionDate'] = pd.to_datetime(wooMinisterie['foi_decisionDate'], format='%Y-%m-%d')
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')

if start_date and end_date:
    min2023extracted = wooMinisterie.loc[(wooMinisterie['foi_decisionDate'] >= start_date) & (wooMinisterie['foi_decisionDate'] <= end_date)]
else:
    min2023extracted = wooMinisterie.loc[(wooMinisterie['foi_decisionDate'] >= '2023-01-01') & (wooMinisterie['foi_decisionDate'] <= '2023-12-31')]

print(f"besluiten van {start_date} tot {end_date}")

besluiten van 2019-04-01 00:00:00 tot 2024-01-10 00:00:00


In [26]:
print(min2023extracted[min2023extracted.index.get_level_values(1) == 1].shape)

(1507, 50)


In [27]:
# extract dates functie uit notebook van Maarten Marx 'DutchDatumParser'

date_label = ['DATE']

def extract_dates(text):
    doc = nlp(text)
    results = [ent.text for ent in doc.ents if ent.label_ in date_label]
    results= {s:dateparser.parse(s, settings={'REQUIRE_PARTS': ['day', 'month', 'year']}) for s in results}
    return results

def sort_dates(dates):
    if dates != None and len(dates) >= 1:
        dates = sorted([x.replace(tzinfo=None) for x in dates.values() if not x is None])
    else:
        return None
    
    # onrealistische datums weghalen
    dates = [x for x in dates if (x.year > 1950 and x.year < 2050)]
    
    if len(dates) == 0 or dates == {}:
        return None
    return dates

In [28]:
# extract alle dates, niet aanbevolen, duurt heel lang
# later in de notebook wordt het alleen op de eerste pagina's uitgevoerd

# min2023extracted['dates'] = min2023extracted['foi_bodyTextOCR'].apply(lambda x: extract_dates(x) if (type(x) == str) else None)
# min2023extracted['dates_sorted'] = min2023extracted['dates'].apply(lambda x: sort_dates(x))min2023extracted

In [29]:
# datums als '2 1 juni 2023' omzetten naar '21 juni 2023' en double whitespace weghalen
def join_numbers(s):
    s = ' '.join(s.split())
    return re.sub(r"(\s?\d)(\s)(\d\s)", '\g<1>\g<3>', s)

# 7 woorden/getallen voor, en 6 woorden/getallen na 'Datum' noteren en een beetje opschonen voor Spacy
def regex_context_dagtekening(s):
    if type(s) == float:
        return None
    else:
        search_result = re.search(r"(\S+\s+){8}Datum[:]?(\s+\S+){6}", s)
        if search_result != None:
            result = search_result[0].translate(str.maketrans('', '', string.punctuation.replace('-', ''))).lower()
            return join_numbers(result)
        else:
            return None
        
def parse_and_combine_dt_dtspacy(row):
    dt_regex = row['dagtekening_extracted']
    dt_spacy = row['dagtekening_spacy_sorted']
    if dt_spacy != None:
        if len(dt_spacy) > 1:
            dt_spacy = max(dt_spacy)
        else:
            dt_spacy = dt_spacy[0]
    if isinstance(dt_regex, datetime):
        if dt_regex.year > datetime.today().year:
            return dt_regex.ctime()
        else:
            return dt_regex
    elif isinstance(dt_spacy, datetime):
        if dt_spacy.year > datetime.today().year:
            return dt_spacy.ctime()
        else:
            return dt_spacy
    else:
        return None

def return_date(s):
    if isinstance(s, str):
        dates = extract_dates(join_numbers(s))
        if len(dates) != 0:
            date_list = [date for date in dates.values() if date != None]
            if len(date_list) >= 1:
                return max(date_list)
        else:
            return None
    else:
        return None
    

In [ ]:
# dagtekening met reguliere expressies eruit halen
min2023extracted['dagtekening'] = min2023extracted['foi_bodyTextOCR'].str.extract(r'(?<=\bDatum\s)(\d+ \w+ \d+)')
min2023extracted['dagtekening_colon'] = min2023extracted['foi_bodyTextOCR'].str.extract(r'(?<=\bDatum:\s)(\d+ \w+ \d+)')
min2023extracted['dagtekening_dashed'] = min2023extracted['foi_bodyTextOCR'].str.extract(r'(?<=\bDatum\s)(\d+-\w+-\d+)')
min2023extracted['dagtekening_before'] = min2023extracted['foi_bodyTextOCR'].str.extract(r'([0-9]? \d+ \w+ \d+)(?=\s+Datum)')
min2023extracted['dagtekening_context'] = min2023extracted['foi_bodyTextOCR'].apply(lambda x: regex_context_dagtekening(x))

# verschillende reguliere expressies combineren tot 1 kolom
min2023extracted['dagtekening_combined'] = min2023extracted['dagtekening'].combine_first(min2023extracted['dagtekening_colon']).combine_first(min2023extracted['dagtekening_dashed']).combine_first(min2023extracted['dagtekening_before'])
min2023extracted.drop(['dagtekening', 'dagtekening_colon', 'dagtekening_dashed', 'dagtekening_before'], axis=1, inplace=True)
min2023extracted['dagtekening_extracted'] = min2023extracted['dagtekening_combined'].apply(lambda x: return_date(x))

In [ ]:
# uit de context de lastigere datums met Spacy proberen op te halen, duurt even!
min2023extracted['dagtekening_spacy'] = min2023extracted['dagtekening_context'].apply(lambda x: extract_dates(x) if (type(x) == str) else None)

# Spacy datums sorteren en filteren
min2023extracted['dagtekening_spacy_sorted'] = min2023extracted['dagtekening_spacy'].apply(lambda x: sort_dates(x))


In [ ]:
min2023extracted['dagtekening_complete'] = min2023extracted.apply(lambda x: parse_and_combine_dt_dtspacy(x), axis=1)

min2023extracted['dagtekening_complete'] = min2023extracted['dagtekening_complete'].groupby(level=0).fillna(method='ffill')
min2023extracted['dagtekening_complete'] = min2023extracted['dagtekening_complete'].groupby(level=0).fillna(method='bfill')

In [ ]:
total_documents = len(set(min2023extracted.index.get_level_values(0)))
regex_extractions = min2023extracted['dagtekening_combined'].loc[(min2023extracted.index.get_level_values(0), [1])].isna().sum()
regexAndSpacy_extractions = len(min2023extracted[min2023extracted.index.get_level_values(1) == 1][min2023extracted[['dagtekening_combined', 'dagtekening_spacy_sorted']].isna().all(1)][['dagtekening_combined', 'dagtekening_spacy_sorted']])

print(f"After regex and Spacy extraction we are still missing {regexAndSpacy_extractions} of {total_documents} dates.")

In [ ]:
min2023extracted.to_pickle(path=current_save_path, compression='gzip')
print(f"saved to {current_save_path}")